### INTRODUCTION BY EXAMPLE


基本例子过一下吧，不然 lightGCN 那个还是看不太懂

https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

------------------
Q: How to fix the "ImportError: IProgress not found. Please update jupyter and ipywidgets although it is installed" (每次 import PyG 都会遇到这个问题)

A: https://stackoverflow.com/questions/67998191/importerror-iprogress-not-found-please-update-jupyter-and-ipywidgets-although

In [3]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
%whos

In [1]:
%%script false --no-raise-error

import torch
from torch_geometric.data import Data

# We show a simple example of an unweighted and undirected graph with three nodes and four edges. 
# Each node contains exactly one feature

# create a 2x4 tensor matrix
# Q: 这里好奇怪，要竖着看？ "construct graph connectivity matrices in COO format" 这里 COO format 啥意思，好处是啥？
# A: the tensor defining the source and target nodes of all edges 所以第一行是 source, 第二行是 target
#    4 个 edge 因为是双向的，你看图就只有两个 edge ，但是由于是双向，所以就变成 4 个了
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)

# create a 3x1 tensor matrix
# this is edge feature??
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, 
            edge_index=edge_index)

print(data)

In [3]:
%%script false --no-raise-error

import torch
from torch_geometric.data import Data

# 这边就是用 edge pair 的写法，看的更清楚一些，然后下面有一个 .t().contiguous() 来转回来到 COO 格式
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)


data = Data(x=x, edge_index=edge_index.t().contiguous())

print(data)

In [4]:
#  会得到 ['edge_index', 'x']， 就是说有哪些 key, 有 edge index 和 x 就是 feature of each node
print(data.keys)

# 这个就是 print note feature, 会得到下面这个, 因为上面定义的 x 就是 是那个 node 然后每个 node 是一个 value
# tensor([[-1.],
#         [ 0.],
#         [ 1.]])
print(data['x'])

# 就是打印一下['edge_index', 'x'] 这两个 key
for key, item in data:
    print(f'{key} => {item}')


'edge_attr' in data


data.num_nodes


data.num_edges


data.num_node_features


data.has_isolated_nodes()


data.has_self_loops()


data.is_directed()


# Transfer data object to GPU.
# device = torch.device('cuda')
# data = data.to(device)

NameError: name 'data' is not defined

In [8]:
# 接下来挺 straightforward 先跳过

_--------------- My wild quest into scipy: try sparse tensor just to see what it looks like ----_

In [9]:
from torch_sparse import SparseTensor, matmul

In [10]:
edge_index = torch.tensor([[0, 1],
                           [0, 1]], dtype=torch.long)

In [11]:
train_sparse_edge_index = SparseTensor(row=edge_index[0], 
                                       col=edge_index[1], 
                                       sparse_sizes=(4, 4))


In [12]:
# 看不出来...
# nnz mean number non zero elements
print(train_sparse_edge_index)

SparseTensor(row=tensor([0, 1]),
             col=tensor([0, 1]),
             size=(4, 4), nnz=2, density=12.50%)


In [13]:
print(train_sparse_edge_index.size(1))

4


In [16]:
print(train_sparse_edge_index)

SparseTensor(row=tensor([0, 1]),
             col=tensor([0, 1]),
             size=(4, 4), nnz=2, density=12.50%)


In [15]:
type(train_sparse_edge_index.to_scipy())

/opt/anaconda3/envs/pyg/lib/python3.7/site-packages/torch_sparse/storage.py:14: UserWarning: `layout` argument unset, using default layout "coo". This may lead to unexpected behaviour.
  warnings.warn('`layout` argument unset, using default layout '


scipy.sparse.coo.coo_matrix

In [17]:
spi_edge_index = train_sparse_edge_index.to_scipy()

In [21]:
# todense 这个 API 真多挺好用，看着很方便, 
# https://scipy-lectures.org/advanced/scipy_sparse/coo_matrix.html
# 不过这个不像是 adjescent matrix?? 这个是 directional 的 representation 呀, row 是 src/from, col 是 dest/to
spi_edge_index.todense()

matrix([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)

In [ ]:
# 试一下 scipy adjescent matrix

In [30]:
import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.sparse import csr_matrix, coo_matrix

arr = np.array([
  [0, 1, 2],
  [1, 0, 0],
  [2, 0, 0]
])

newarr = coo_matrix(arr)

print(connected_components(newarr))

(1, array([0, 0, 0], dtype=int32))


In [31]:
print(newarr)

  (0, 1)	1
  (0, 2)	2
  (1, 0)	1
  (2, 0)	2


In [25]:
# 诶，这不是一样的吗？所以 todense 其实就是 print 出 adjescent matrix
# adjescent matrix 是有向的
print(newarr.todense())

[[0 1 2]
 [1 0 0]
 [2 0 0]]


In [32]:
print(-(1 << 10))

-1024
